In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

class RidgeRegression:
    def __init__(self, tau: float = 1.0) -> None:
        self.tau = tau
        self.weights_ = None
        self.svd_ = None

    def _compute_svd(self, X: np.ndarray) -> None:
        """Вычисление сингулярного разложения"""
        self.svd_ = np.linalg.svd(X, full_matrices=False)

    def fit(self, X: np.ndarray, y: np.ndarray, tau: float | None = None) -> 'RidgeRegression':
        """Обучение модели"""
        if not self.svd_:
            self._compute_svd(X)
        if tau is not None:
            self.tau = tau

        U, S, Vt = self.svd_
        S_inv = np.diag(S / (S ** 2 + self.tau))
        self.weights_ = Vt.T @ S_inv @ U.T @ y
        return self

    def predict(self, X: np.ndarray) -> np.array:
        """Предсказание с использованием обученной модели"""
        return X @ self.weights_

    def quality(self, X: np.ndarray, y: np.ndarray) -> np.floating:
        """Оценка качества модели"""
        return np.linalg.norm(X @ self.weights_ - y) ** 2


# Загрузка данных
df = pd.read_csv('data.csv')

X = df[['Socioeconomic Score', 'Study Hours', 'Sleep Hours', 'Attendance (%)']].values
y = df['Grades'].values

# Стандартизация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Подбор оптимального параметра регуляризации через кросс-валидацию
def optimal_lambda(X, y, lambdas, n_splits=5):
    best_lambda = None
    best_mse = float('inf')
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for lambda_param in lambdas:
        mse_list = []
        
        for train_index, val_index in kf.split(X):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            
            # Обучение модели
            model = RidgeRegression(tau=lambda_param)
            model.fit(X_train, y_train)
            
            # Предсказания на валидационном наборе
            y_pred = model.predict(X_val)
            mse = mean_squared_error(y_val, y_pred)
            mse_list.append(mse)
        
        avg_mse = np.mean(mse_list)
        
        if avg_mse < best_mse:
            best_mse = avg_mse
            best_lambda = lambda_param
    
    return best_lambda

# Список значений для λ
lambdas = np.logspace(-5, 5, 100)

# Поиск оптимального λ
optimal_lambda_value = optimal_lambda(X_scaled, y, lambdas)
print(f"Оптимальный параметр регуляризации λ: {optimal_lambda_value}")

# Обучение модели с оптимальным λ
model = RidgeRegression(tau=optimal_lambda_value)
model.fit(X_scaled, y)

# Проверка с помощью библиотеки sklearn (Ridge)
from sklearn.linear_model import Ridge as SklearnRidge
ridge = SklearnRidge(alpha=optimal_lambda_value)
ridge.fit(X_scaled, y)
sklearn_coefficients = ridge.coef_

print("Коэффициенты из sklearn Ridge:", sklearn_coefficients)

# Коэффициенты из нашей реализации
print("Коэффициенты из нашей реализации:", model.weights_)


Оптимальный параметр регуляризации λ: 93.26033468832199
Коэффициенты из sklearn Ridge: [ 2.9453858   7.40347015  0.11190459 -0.43853676]
Коэффициенты из нашей реализации: [ 2.9453858   7.40347015  0.11190459 -0.43853676]
